In [7]:
from dask_yarn import YarnCluster
from dask.distributed import Client
import dask.array as da
import dask.dataframe as dd
import numpy as np
from google.cloud import bigquery

cluster = YarnCluster()
client = Client(n_workers=3, threads_per_worker=4)

# cluster.adapt() # Dynamically scale Dask resources

In [8]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 3
Total threads: 6,Total memory: 14.65 GiB
Comm: tcp://127.0.0.1:43269,Workers: 3
Dashboard: http://127.0.0.1:8787/status,Total threads: 6
Started: Just now,Total memory: 14.65 GiB
Comm: tcp://127.0.0.1:33309,Total threads: 2
Dashboard: http://127.0.0.1:34849/status,Memory: 4.88 GiB
Nanny: tcp://127.0.0.1:36907,


In [ ]:
# Export data to gcs 

client = bigquery.Client()
bucket_name = 'explore-gcp-v1'
project = "publicdata"
dataset_id = "samples"
table_id = "natality"

destination_uri = "gs://{}/data/{}".format(bucket_name, "natality-*.csv")
dataset_ref = bigquery.DatasetReference(project, dataset_id)
table_ref = dataset_ref.table(table_id)

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

In [9]:
datatypes = dtype={'child_race': 'float64',
       'father_race': 'float64',
       'mother_race': 'float64'}


df = dd.read_csv("gcs://explore-gcp-v1/data/natality-*.csv", dtype=datatypes)

In [10]:
len(client.scheduler_info()['workers'])

3

In [11]:
df.count().compute()

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
### preprocessing 
df_natality_select = df[(df.weight_pounds.notnull()) & (df.mother_age.notnull()) & (df.father_age.notnull()) & (df.gestation_weeks.notnull()) & (df.weight_gain_pounds.notnull()) & (df.apgar_5min.notnull())] \
                    [["weight_pounds", "mother_age", "father_age", "gestation_weeks", "weight_gain_pounds", "apgar_5min"]]

In [ ]:
%%time
df_natality_select.count().compute()